In [1]:
import hashlib
import numpy as np
import pandas as pd
import csv
import pinyin
from copy import deepcopy

In [ ]:
# Assign hash key to AWE-CSV file
# Then assign hash key to other CSV files
# Use hash key to search in between

In [2]:
# Name List | READ
row_list = []
with open("source-csv/AWE-CSV.csv", 'rb') as csvfile:
    read_row = csv.reader(csvfile, delimiter=',')
    for row in read_row:
        row_list.append(row)

In [3]:
# Name list | Modify
# Add hash to English name
deli = ' - '
new_row_list = deepcopy(row_list[1:])  # Copy by value 
for row in new_row_list:
    tmp = hashlib.sha1(row[1].lower() + deli + row[2].lower()).hexdigest()
    row.insert(3, tmp)

In [4]:
# Name list | Write
# Write Hashed English namefile to newfile
new_title = deepcopy(row_list[0])
new_title.insert(3, 'Hash_Key')
with open('output-csv/HASH-AWE-CSV.csv', 'wb') as f:
    writer = csv.writer(f)
    writer.writerow(new_title)
    writer.writerows(new_row_list)

In [5]:
# Convert chinese name into pinyin with delimiter then return sha-1 hash
def chinese_name_to_lower_pinyin_hash(c_name, deli=' - '):
    sep     = '·'
    sep_loc = c_name.find(sep)
    if sep_loc == -1:
        sep     = '•'
        sep_loc = c_name.find(sep)
    if sep_loc == -1:
        l_name, f_name = c_name[:3], c_name[3:]
    else:
        l_name, f_name = c_name.split(sep)
    l_name_pinyin = pinyin.get(l_name, format="", delimiter="")
    f_name_pinyin = pinyin.get(f_name, format="strip", delimiter="")    
    hash_key = l_name_pinyin.lower() + deli + f_name_pinyin.lower()
    hashed_name = hashlib.sha1(hash_key).hexdigest()
    return hashed_name
def get_rid_of_paranthesis(c_name):
    left_pos = c_name.find('（')
    if left_pos != -1:
        right_pos = c_name.find('）')
#         print right_pos, len(c_name)
        c_name = c_name[:left_pos] + c_name[right_pos+3:] if right_pos != len(c_name) else c_name[:left_pos]
    return c_name

In [6]:
# For .csv files in 'source-csv' folder, add hashkey column right after their name column
def chinese_to_hash(in_file_name, out_file_name, name_pos):
    # file | Read
    raw_list = []
    with open(in_file_name, 'rb') as csvfile:
        read_row = csv.reader(csvfile, delimiter=',')
        for row in read_row:
            raw_list.append(row)
    # file | Modify
    new_list = []
    deli = ' - '
    for row in raw_list[2:]:
        tmp = row[:name_pos+1] + [chinese_name_to_lower_pinyin_hash(get_rid_of_paranthesis(row[name_pos]), deli)] + row[name_pos+1:]
        new_list.append(tmp)
    # file | Write
    new_title_1 = deepcopy(raw_list[0])
    new_title_2 = deepcopy(raw_list[1])
    new_title_1.insert(name_pos+1, 'Hash_Key')
    new_title_2.insert(name_pos+1, '搜索键')
    with open(out_file_name, 'wb') as f:
        writer = csv.writer(f)
        writer.writerow(new_title_1)
        writer.writerow(new_title_2)
        writer.writerows(new_list)    

In [7]:
chinese_to_hash('source-csv/City.csv', 'output-csv/HASH-CITY.csv', 6)

In [8]:
chinese_to_hash('source-csv/County.csv', 'output-csv/HASH-COUNTY.csv', 7)

In [9]:
chinese_to_hash('source-csv/Province-EDU.csv', 'output-csv/HASH-PROVINCE-EDU.csv', 0)

In [10]:
chinese_to_hash('source-csv/Province-WORK.csv', 'output-csv/HASH-PROVINCE-WORK.csv', 0)